In [95]:
# add ../oepul_chat to path 
from llama_index.evaluation import RetrieverEvaluator
import sys
from pprint import pprint
import re
sys.path.append('../oepul_chat')

In [2]:
from oepul_chat import load_index, get_header_path_mappings

# Load summary index

In [144]:
def get_filename(doc_id, summary_index):
    filename =  summary_index.ref_doc_info[doc_id].metadata['File Name']
    
    # Split the filename by underscores and hyphens
    parts = re.split('[_-]', filename)

    # Remove numeric parts from each part of the filename
    parts = [re.sub(r'\d+', '', part) for part in parts]

    # drop empty elements
    parts = [part for part in parts if part]

    # Reconstruct the filename with underscores
    new_filename = '-'.join(parts)

    # Remove ".pdf" extension
    new_filename = new_filename.replace('.pdf', '')

    # Remove zero at beginning of filename
    new_filename = new_filename.lstrip('O').strip("-")

    return new_filename

In [141]:
summary_index = load_index('indices/summary_index/')
documents = summary_index.docstore.docs
[doc.text for doc in list(documents.values())][0]

'## ÜBERSICHT\nDie Prämie wird für Almweideflächen gewährt, die mit Tieren bestoßen werden. Gefördert werden Kosten und Einkommensverluste, die durch die Einhaltung der Verpflichtungen, insbesondere durch den höheren Arbeitszeitbedarf für Weidepflege und den Verzicht auf Mineraldünger sowie chemischen Pflanzenschutz entstehen. Optional erfolgt ein Prämienzuschlag für naturschutzfachlich begründete Auflagen. Die Maßnahme ist von der Almbewirtschafterin oder dem Almbewirtschafter zu beantragen.\n\n## ZIELSETZUNG\nDie Maßnahme dient dem Erhalt der Kulturlandschaft und dem Schutz der Biodiversität durch standortangepasste Land- und Forstwirtschaft. Durch Forcierung der Berücksichtigung gebietsspezifischer Aspekte trägt die Maßnahme zur Erreichung der Biodiversitätsziele bei. Die Maßnahme soll außerdem zur Verbesserung des Oberflächen- und Grundwasserschutzes beitragen.\n\n## TEILNAHMEVORAUSSETZUNGEN\n\n\n## VERTRAGSZEITRAUM\nDer Verpflichtungs- und Vertragszeitraum der Maßnahme beträgt ink

In [145]:
# Get summaries from index
doc_ids = list(summary_index.ref_doc_info.keys())
summaries = [
    f'Datei: {get_filename(doc_id, summary_index)}\nInhalt: {summary_index.get_document_summary(doc_id)}'
    for doc_id in doc_ids]
summary_len = len(" ".join(summaries))
print(f'Summary total length: {summary_len}')

Summary total length: 22713


In [143]:

text = "\n\n ---- \n\n".join(summaries)
# write to file
with open('tmp/summary.txt', 'w') as f:
    f.write(text)


In [65]:
index = load_index('indices/oepul_index/')
normal_index_len = len(" ".join([doc.text for doc in list(documents.values())]))
print(f'Normal index total length: {normal_index_len}')
print(f'Summary index to normal ratio: {summary_len / normal_index_len }')

Normal index total length: 503984
Summary index to normal ratio: 0.04356090669545065


In [30]:
pprint(vars(documents['6ab3976a-0248-442d-aba4-a20ed2d3a307']))

{'embedding': None,
 'end_char_idx': 3023,
 'excluded_embed_metadata_keys': ['file_name',
                                  'file_type',
                                  'file_size',
                                  'creation_date',
                                  'last_modified_date',
                                  'last_accessed_date'],
 'excluded_llm_metadata_keys': ['file_name',
                                'file_type',
                                'file_size',
                                'creation_date',
                                'last_modified_date',
                                'last_accessed_date'],
 'id_': '6ab3976a-0248-442d-aba4-a20ed2d3a307',
 'metadata': {'Content Type': 'text',
              'File Name': 'O6_14_Almbewirtschaftung_2023_04.pdf'},
 'metadata_seperator': '\n',
 'metadata_template': '{key}: {value}',
 'relationships': {<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b3487c8e-0061-4cdf-a9c9-b1d203c892e0', node_type=<ObjectType

In [28]:
documents

{'6ab3976a-0248-442d-aba4-a20ed2d3a307': TextNode(id_='6ab3976a-0248-442d-aba4-a20ed2d3a307', embedding=None, metadata={'File Name': 'O6_14_Almbewirtschaftung_2023_04.pdf', 'Content Type': 'text'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b3487c8e-0061-4cdf-a9c9-b1d203c892e0', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'File Name': 'O6_14_Almbewirtschaftung_2023_04.pdf', 'Content Type': 'text'}, hash='1fec3828f0763f1763f51e22e20c3ffa3c8947cc9cab21c95629ec4d7867f95a'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='68e4e5d2-33db-4760-a5e5-5f8ccbc1e185', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='40f6b561abd10f6cb5effc279dbae55663f555b6a33b7ff1696240111eaa8965')}, text='## ÜBERSI

# Response Evaluator 

In [12]:
from llama_index.evaluation import RelevancyEvaluator

evaluator = RelevancyEvaluator()

In [10]:
# Load index
index = load_index('indices/oepul_index/')

# Get query engine from index
query_engine = index.as_query_engine(similarity_top_k=8)

query = "Was sind die unterschiedliche Anforderungen an abfrierenden und winterharte in der Maßnahme Zwischenfruchtanbau?"
response = query_engine.query(query)

In [18]:
import asyncio
import nest_asyncio
nest_asyncio.apply()


eval_result = asyncio.run(evaluator.aevaluate_response(query=query, response=response))
pprint(vars(eval_result))

{'contexts': None,
 'feedback': 'YES',
 'invalid_reason': None,
 'invalid_result': False,
 'pairwise_source': None,
 'passing': True,
 'query': 'Was sind die unterschiedliche Anforderungen an abfrierenden und '
          'winterharte in der Maßnahme Zwischenfruchtanbau?',
 'response': 'Die unterschiedlichen Anforderungen an abfrierende und '
             'winterharte Pflanzen im Rahmen der Maßnahme Zwischenfruchtanbau '
             'sind, dass die Mischungen sowohl aus abfrostenden als auch aus '
             'winterharten Arten bestehen dürfen. Für die Begrünungsvarianten '
             '1 bis 5 und 7 sind sowohl abfrierende als auch winterharte '
             'Zwischenfrüchte erlaubt, während für die Variante 6 nur '
             'bestimmte winterharte Zwischenfrüchte zugelassen sind.',
 'score': 1.0}


# Retriever Evaluator

In [5]:
index = load_index('indices/oepul_index/')
retriever = index.as_retriever(top_k=8)
documents = index.docstore.docs

header_path_mappings = get_header_path_mappings(documents)

# each element in the list as line in the file
with open('tmp/header_path_mappings.txt', 'w') as f:
    for key in header_path_mappings:
        f.write(f'{key}\n')

In [35]:
def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()
    columns = {"retrievers": [name], "hit_rate": [hit_rate], "mrr": [mrr]}

    if include_cohere_rerank:
        crr_relevancy = full_df["cohere_rerank_relevancy"].mean()
        columns.update({"cohere_rerank_relevancy": [crr_relevancy]})

    metric_df = pd.DataFrame(columns)

    return metric_df

In [22]:
def get_ids_of_header_path(header_path, header_path_mappings):
    return header_path_mappings[header_path]


get_ids_of_header_path(
    'Vorbeugender Grundwasserschutz – Acker/FÖRDERBEDINGUNGEN/SCHLAGBEZOGENE AUFZEICHNUNGEN',
    header_path_mappings)

['6193a3bc-df78-4480-b945-21c523890867']

In [27]:
def get_doc_ids_by_query(query):
    res = retriever.retrieve(query)
    return [r.id_ for r in res]

get_doc_ids_by_query("Wie groß müssen die Schläge in der Maßnahme 'Vorbeugender Grundwasserschutz sein?")

['577cb7ff-6fd1-425a-bdcd-099455def520',
 '02a0e1b7-6238-4ea7-888e-11815d6e7a49']

In [30]:
eval = [{
    "query": "Wie groß müssen die Schläge in der Maßnahme 'Vorbeugender Grundwasserschutz sein?",
    "header_paths": [
        'Vorbeugender Grundwasserschutz – Acker/FÖRDERBEDINGUNGEN/SCHLAGBEZOGENE AUFZEICHNUNGEN'],
    "expected_answer": "Schläge müssen mindestens 0,3 ha groß sein."
}]

for item in eval: 
    query = item['query']
    expected_header_paths = item['header_paths']
    expected_answer = item['expected_answer']
    
    true_doc_ids = []
    for header_path in expected_header_paths:
        true_doc_ids.extend(get_ids_of_header_path(header_path, mappings))

    
    
    retriever_evaluator.evaluate(
        query=query, expected_ids=true_doc_ids
    )
    

NameError: name 'retriever_evaluator' is not defined